In [ ]:
#hexid*time period -
step1- classify each hex*TP as high,mid,low MR based on percentage of orders. (if ME is 40, 35-45,)
        what was there in pre period - AOR,Gross orders, 
    
aor,g2n,mr pre period bucket

then
for pre and post - we get no gross orders,APR,APR, G2,cobra,Mapped_exp by lm buckets 


--
    

In [ ]:
# In test hexes which were the ones with most acceptance beyond 1.5KMs. And identify the difference in Avg. AOR pre/post.

# Need to look at Hex, test/control, Time_Period Level

# Orders

# G2N

# AOR

# MR - Avg Mapped Riders

# MR composition by FM

# Which FM the order is accepted (0-500m, 500m-1KM, 1KM-2KM, 2KM+)

# MR composition by Captains segments



In [93]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [157]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        #host="presto.processing.yoda.run",
        #host ="prime-trino-2.serving.data.production.internal",
        #host = "processing-2.processing.data.production.internal",
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [32]:
# query="""
# select 
#         order_id, yyyymmdd, city_name, order_type, captain_id, pickup_location_hex_8,map_riders,
#         event_type, order_status, spd_fraud_flag, service_obj_service_name as service_name,
#         accept_to_pickup_distance, estimated_accept_to_pickup_distance as fm_osrm, eta,
#         distance_final_distance, updated_hhmmss, updated_epoch, dispatch_global_priority,

#         (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
#         when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
#         when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
#         when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
#         else 'Rest_Evening' end) as time_bucket,
#         row_number() over (partition by order_id order by updated_epoch desc) as rank_1,

#         cardinality(array_remove(array_distinct(array_agg(captain_id) over (partition by order_id)), '')) as captains_pinged,
#         array_agg(event_type) over (partition by order_id) as event_type_agg,
#         cardinality(array_distinct(array_remove(split(replace(replace(replace(map_riders, '['), ']'), '"'), ','), ''))) as map_riders_count, cancel_reason


# from 
#         orders.order_logs_immutable
# where 
#         yyyymmdd = '20230710'
#         and city_name in ('Hyderabad')
#         and service_obj_service_name in ('Auto')and order_id='64aae4733cb7d44d0b1a76a9'
# """

# v1=pd.read_sql_query(query, metabase_connection)


In [58]:

query="""
        select yyyymmdd,test_ctrl,time_bucket,pickup_location_hex_8,order_id,
        
        count(distinct map_riders_lst) as total_map_riders,
        count(distinct case when fm<=500 then map_riders_lst end) as MR_within_500m,
        count(distinct case when fm>500 and fm<=1000 then map_riders_lst end) as MR_500m_to_1km,        
        count(distinct case when fm>1000 and fm<=2000 then map_riders_lst end) as MR_1km_to_2km,
        count(distinct case when fm>2000 then map_riders_lst end) as MR_more_than_2km,

        count(distinct case when performance='UHP' then map_riders_lst end) as MR_UHP_caps,
        count(distinct case when performance='HP' then map_riders_lst end) as MR_HP_caps,
        count(distinct case when performance='MP' then map_riders_lst end) as MR_MP_caps,
        count(distinct case when performance='LP' then map_riders_lst end) as MR_LP_caps,

        count(distinct case when contains(event_type_agg, 'accepted') and fm<=500 then order_id end) accp_within_500m_fm,
        count(distinct case when contains(event_type_agg, 'accepted') and fm<=1000 then order_id end) accp_within_500m_to_2km_fm,
        count(distinct case when contains(event_type_agg, 'accepted') and fm<=2000 then order_id end) accp_within_1_2km_fm, 
        count(distinct case when contains(event_type_agg, 'accepted') and fm>2000 then order_id end) accp_with_more_than_2km_fm        
        
        from
        (
            select tt1.yyyymmdd,test_ctrl,pickup_location_hex_8,tt1.time_bucket,tt1.order_id,map_riders_lst,fm,performance,event_type_agg
            from
            (
              select distinct yyyymmdd,time_bucket,pickup_location_hex_8,order_id,map_riders_lst
                
              from 
                (
                    select yyyymmdd,pickup_location_hex_8,order_id,
                    cast(json_parse(map_riders) as array(varchar)) as map_riders,
                    
                   (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
                    when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
                    when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
                    when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
                    else 'Rest_Evening' end) as time_bucket

                    from orders.order_logs_immutable
                    where 
                    yyyymmdd = '20230710'
                    and city_name in ('Hyderabad')
                    and service_obj_service_name in ('Auto')and order_id='64aae4733cb7d44d0b1a76a9'

                ) 
                cross join unnest(map_riders) as t(map_riders_lst)
                
            )as tt1

            left join(
                      
                    select distinct yyyymmdd,order_id,captain_id,
                    array_agg(event_type) over (partition by order_id,captain_id) as event_type_agg
                    
                    from orders.order_logs_immutable
                    where 
                    yyyymmdd = '20230710'
                    and city_name in ('Hyderabad')
                    and service_obj_service_name in ('Auto')and order_id='64aae4733cb7d44d0b1a76a9'

                    ) nt1
                    
                    on tt1.yyyymmdd=nt1.yyyymmdd and tt1.order_id = nt1.order_id and tt1.map_riders_lst = nt1.captain_id
                    
            
            left join (

                    select yyyymmdd,captain_id,order_id,      
                            avg(captains_firstmiledistance) as fm
                            from g2n.flattened_captains
                            where yyyymmdd ='20230710'
                            and service_detail_id in ('5ef2bc5b85846b775f97d170' ,'61769b50d21e19252c6f60e8')
                            and order_id='64aae4733cb7d44d0b1a76a9'
                        group by 1,2,3 
                    ) as tt2

                    on tt1.map_riders_lst = tt2.captain_id and tt1.order_id = tt2.order_id and tt1.yyyymmdd=tt2.yyyymmdd
            left join (

                        select captain_id,performance 
                        from hive.datasets_internal.captain_auto_pc_segments_v2 
                        where yyyymmdd='20230724'
                    ) tt3
                    on tt1.map_riders_lst = tt3.captain_id


        left join (
                    select city_name,test_ctrl, hex_id ,time_bucket
                    from experiments_internal.auto_apr_fm_reduction_exp_230704 
                    where city_name in ('Pune','Hyderabad')

                 ) exp_dt
                 on tt1.pickup_location_hex_8 = exp_dt.hex_id and tt1.time_bucket = exp_dt.time_bucket


 
          where case when tt1.yyyymmdd >='20230710' and concat(tt1.pickup_location_hex_8,tt1.time_bucket) 
               in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                    '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                    '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                    '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                    '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                    '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                    '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                    '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                    '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                    '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                    '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                    '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                    '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                    '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                    '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                    '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                    '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                    '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                    '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                    '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                    '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                    '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                    '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                    '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                    '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                    '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                    '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                    '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                    '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                    '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                    '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                    '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                    '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                    '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                    '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                    '8860a24a65fffffmorning_peak'
                    
                    ) then 1 else 0 end =0
                
                and 
                   concat(tt1.time_bucket,tt1.pickup_location_hex_8) not in 
                    ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                    'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                    'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                    'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                    'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                    'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                    'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                    'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                    'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                    )
            
            



      )
      group by 1,2,3,4,5 
        
"""
k1=pd.read_sql_query(query, metabase_connection)


In [56]:
k1.to_clipboard()

In [60]:
k1

,yyyymmdd,test_ctrl,time_bucket,pickup_location_hex_8,order_id,total_map_riders,MR_within_500m,MR_500m_to_1km,MR_1km_to_2km,MR_more_than_2km,MR_UHP_caps,MR_HP_caps,MR_MP_caps,MR_LP_caps,accp_within_500m_fm,accp_within_500m_to_2km_fm,accp_within_1_2km_fm,accp_with_more_than_2km_fm
0,20230710,ctrl_4,rest_morning,8860a258b9fffff,64aae4733cb7d44d0b1a76a9,7,1,0,6,0,0,2,1,4,0,0,1,0


In [ ]:
#          case when lm<=3 then '0_3km'
#          when lm<=5 then '3_5km'
#          when lm<=8 then '5_8km'
#          when lm<=12 then '8_12km'
#          when lm>12 then '12km_above'
#          end as lm_bucket,

In [95]:
#final

In [ ]:
## AOR,

##data needed - AOR,APR,G2N,MR

In [131]:
per_Days_data=pd.DataFrame()

for i in pd.date_range('20230704','20230723'):

    query="""

with set_one as
    (
    select yyyymmdd,city_name,test_ctrl,pickup_location_hex_8,time_bucket,
    count(distinct order_id) as "Gross_Orders",
    count(distinct case when order_status = 'dropped' and spd_fraud_flag != True then order_id end) as "Net_Orders",
    count(distinct case when contains(event_type_agg, 'accepted') then order_id end) as "accepted_orders"  
from

(
    select   yyyymmdd,city_name,test_ctrl,pickup_location_hex_8,time_bucket,order_id,order_status,spd_fraud_flag,event_type_agg
            
    from        
    (

    with  order_filtering as 
        (
            select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,distance_final_distance as LM,
                    CASE 
                        WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                        WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                        WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                        WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                        ELSE 'rest_evening' END AS time_period

            from 
                orders.order_logs_snapshot
                -- where yyyymmdd = '20230625' 
                where yyyymmdd = cast({dt} as varchar)
                and service_obj_service_name = 'Auto'
                and lower(trim(city_name)) in ('hyderabad','pune') 
        ),
    v0 as (
            select *
            from        
                    (select 
                            order_id, yyyymmdd, city_name, order_type, captain_id, pickup_location_hex_8,
                            event_type, order_status, spd_fraud_flag, service_obj_service_name as service_name,
                            accept_to_pickup_distance, estimated_accept_to_pickup_distance as fm_osrm, eta,
                            distance_final_distance, updated_hhmmss, updated_epoch, dispatch_global_priority,
                            

                            (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
                            when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
                            when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
                            when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
                            else 'Rest_Evening' end) as time_bucket,
                            row_number() over (partition by order_id order by updated_epoch desc) as rank_1,

                            cardinality(array_remove(array_distinct(array_agg(captain_id) over (partition by order_id)), '')) as captains_pinged,
                            array_agg(event_type) over (partition by order_id) as event_type_agg,
                            cardinality(array_distinct(array_remove(split(replace(replace(replace(map_riders, '['), ']'), '"'), ','), ''))) as map_riders_count, cancel_reason


                    from 
                            orders.order_logs_immutable
                    where 
                            yyyymmdd = '{dt}'
                            and city_name in ('Pune','Hyderabad')
                            and order_id in (select distinct order_id from order_filtering)
                            and service_obj_service_name in ('Auto')
                    )
        ),

        v1 as 
        (
            select city_name,test_ctrl, hex_id ,time_bucket
            from experiments_internal.auto_apr_fm_reduction_exp_230704 
            where city_name in ('Pune','Hyderabad')

        )        
        
    select 
            v0.*, test_ctrl,LM
    from 
            v0 
            left join v1 
            on v0.pickup_location_hex_8=v1.hex_id and v0.time_bucket = v1.time_bucket 
            left join order_filtering
            on v0.order_id = order_filtering.order_id
            
            join (
                   select * from
                   hive.experiments.captain_exposure_to_experiment
               ) as exp
               on V0.captain_id=exp.captain_id
            
            
            
          where case when v0.yyyymmdd >='20230710' and concat(v0.pickup_location_hex_8,v0.time_bucket) 
               in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                    '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                    '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                    '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                    '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                    '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                    '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                    '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                    '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                    '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                    '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                    '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                    '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                    '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                    '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                    '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                    '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                    '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                    '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                    '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                    '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                    '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                    '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                    '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                    '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                    '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                    '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                    '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                    '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                    '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                    '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                    '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                    '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                    '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                    '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                    '8860a24a65fffffmorning_peak'
                    
                    ) then 1 else 0 end =0
                
                and 
                   concat(v0.time_bucket,v0.pickup_location_hex_8) not in 
                    ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                    'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                    'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                    'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                    'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                    'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                    'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                    'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                    'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                    )
            
            
            
        )
    )
    group by 1,2,3,4,5
),
    
set_two as

    (

    select yyyymmdd,test_ctrl,time_bucket,pickup_location_hex_8,
    
    sum(total_map_riders) as total_map_riders,
    avg(total_map_riders) as avg_map_riders,
    
    sum(MR_within_500m) as total_MR_within_500m,
    avg(MR_within_500m) as avg_MR_within_500m,
    
    sum(MR_500m_to_1km) as total_MR_500m_to_1km,
    avg(MR_500m_to_1km) as avg_MR_500m_to_1km,
    
    sum(MR_1km_to_2km) as total_MR_1km_to_2km,
    avg(MR_1km_to_2km) as avg_MR_1km_to_2km,    
    
    sum(MR_more_than_2km) as total_MR_more_than_2km,
    avg(MR_more_than_2km) as avg_MR_more_than_2km,
    
    sum(MR_UHP_caps) as total_MR_UHP_caps,
    avg(MR_UHP_caps) as avg_MR_UHP_caps,
    
    sum(MR_HP_caps) as total_MR_HP_caps,
    avg(MR_HP_caps) as avg_MR_HP_caps,    
    
    sum(MR_MP_caps) as total_MR_MP_caps,
    avg(MR_MP_caps) as avg_MR_MP_caps,
    
    sum(MR_LP_caps) as total_MR_LP_caps,
    avg(MR_LP_caps) as avg_MR_LP_caps,
    
    sum(accp_within_500m_fm) as total_accp_within_500m_fm,
    avg(accp_within_500m_fm) as avg_accp_within_500m_fm,
    
    sum(accp_within_500m_to_1km_fm) as total_accp_within_500m_to_1km_fm,
    avg(accp_within_500m_to_1km_fm) as avg_accp_within_500m_to_1km_fm,
    
    sum(accp_within_1_2km_fm) as total_accp_within_1_2km_fm,
    avg(accp_within_1_2km_fm) as avg_accp_within_1_2km_fm,
    
    sum(accp_with_more_than_2km_fm) as total_accp_with_more_than_2km_fm,
    avg(accp_with_more_than_2km_fm) as avg_accp_with_more_than_2km_fm
       
       from
       (
       


        select yyyymmdd,test_ctrl,time_bucket,pickup_location_hex_8,order_id,
        
        count(distinct map_riders_lst) as total_map_riders,
        count(distinct case when fm<=500 then map_riders_lst end) as MR_within_500m,
        count(distinct case when fm>500 and fm<=1000 then map_riders_lst end) as MR_500m_to_1km,        
        count(distinct case when fm>1000 and fm<=2000 then map_riders_lst end) as MR_1km_to_2km,
        count(distinct case when fm>2000 then map_riders_lst end) as MR_more_than_2km,

        count(distinct case when performance='UHP' then map_riders_lst end) as MR_UHP_caps,
        count(distinct case when performance='HP' then map_riders_lst end) as MR_HP_caps,
        count(distinct case when performance='MP' then map_riders_lst end) as MR_MP_caps,
        count(distinct case when performance='LP' then map_riders_lst end) as MR_LP_caps,

        count(distinct case when contains(event_type_agg, 'accepted') and fm<=500 then order_id end) accp_within_500m_fm,
        count(distinct case when contains(event_type_agg, 'accepted') and fm<=1000 then order_id end) accp_within_500m_to_1km_fm,
        count(distinct case when contains(event_type_agg, 'accepted') and fm<=2000 then order_id end) accp_within_1_2km_fm, 
        count(distinct case when contains(event_type_agg, 'accepted') and fm>2000 then order_id end) accp_with_more_than_2km_fm        
        
        from
        (
            select tt1.yyyymmdd,test_ctrl,pickup_location_hex_8,tt1.time_bucket,tt1.order_id,map_riders_lst,fm,performance,event_type_agg
            from
            (
              select distinct yyyymmdd,time_bucket,pickup_location_hex_8,order_id,map_riders_lst
                
              from 
                (
                    select yyyymmdd,pickup_location_hex_8,order_id,
                    cast(json_parse(map_riders) as array(varchar)) as map_riders,
                    
                   (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
                    when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
                    when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
                    when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
                    else 'Rest_Evening' end) as time_bucket

                    from orders.order_logs_immutable
                    where 
                    yyyymmdd = cast({dt} as varchar)
                    and city_name in ('Hyderabad','Pune')
                    and service_obj_service_name in ('Auto')

                ) 
                cross join unnest(map_riders) as t(map_riders_lst)
                
            )as tt1

            left join(
                      
                    select distinct yyyymmdd,order_id,captain_id,
                    array_agg(event_type) over (partition by order_id,captain_id) as event_type_agg
                    
                    from orders.order_logs_immutable
                    where 
                    yyyymmdd = cast({dt} as varchar)
                    and city_name in ('Hyderabad','Pune')
                    and service_obj_service_name in ('Auto')

                    ) nt1
                    
                    on tt1.yyyymmdd=nt1.yyyymmdd and tt1.order_id = nt1.order_id and tt1.map_riders_lst = nt1.captain_id
                    
            
            left join (

                    select yyyymmdd,captain_id,order_id,      
                            avg(captains_firstmiledistance) as fm
                            from g2n.flattened_captains
                            where yyyymmdd = cast({dt} as varchar)
                            and service_detail_id in ('5ef2bc5b85846b775f97d170' ,'61769b50d21e19252c6f60e8')
                        group by 1,2,3 
                    ) as tt2

                    on tt1.map_riders_lst = tt2.captain_id and tt1.order_id = tt2.order_id and tt1.yyyymmdd=tt2.yyyymmdd
            left join (

                        select captain_id,performance 
                        from hive.datasets_internal.captain_auto_pc_segments_v2 
                        where yyyymmdd = cast({dt} as varchar)
                    ) tt3
                    on tt1.map_riders_lst = tt3.captain_id


        left join (
                    select city_name,test_ctrl, hex_id ,time_bucket
                    from experiments_internal.auto_apr_fm_reduction_exp_230704 
                    where city_name in ('Pune','Hyderabad')

                 ) exp_dt
                 on tt1.pickup_location_hex_8 = exp_dt.hex_id and tt1.time_bucket = exp_dt.time_bucket


 
          where case when tt1.yyyymmdd >='20230710' and concat(tt1.pickup_location_hex_8,tt1.time_bucket) 
               in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                    '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                    '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                    '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                    '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                    '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                    '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                    '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                    '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                    '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                    '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                    '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                    '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                    '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                    '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                    '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                    '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                    '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                    '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                    '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                    '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                    '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                    '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                    '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                    '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                    '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                    '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                    '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                    '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                    '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                    '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                    '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                    '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                    '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                    '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                    '8860a24a65fffffmorning_peak'
                    
                    ) then 1 else 0 end =0
                
                and 
                   concat(tt1.time_bucket,tt1.pickup_location_hex_8) not in 
                    ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                    'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                    'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                    'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                    'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                    'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                    'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                    'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                    'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                    )
            
            



          )
          group by 1,2,3,4,5 
   
       )
       group by 1,2,3,4
   )
   
   
   select xz1.yyyymmdd,xz1.city_name,xz1.test_ctrl,xz1.pickup_location_hex_8,xz1.time_bucket,
   Gross_Orders,Net_Orders,accepted_orders,
   total_map_riders,avg_map_riders,total_MR_within_500m,avg_MR_within_500m,
   total_MR_500m_to_1km,avg_MR_500m_to_1km,total_MR_1km_to_2km,avg_MR_1km_to_2km,
   total_MR_more_than_2km,avg_MR_more_than_2km,total_MR_UHP_caps,avg_MR_UHP_caps,
   total_MR_HP_caps,avg_MR_HP_caps,total_MR_MP_caps,avg_MR_MP_caps,total_MR_LP_caps,avg_MR_LP_caps,
   total_accp_within_500m_fm,avg_accp_within_500m_fm,total_accp_within_500m_to_1km_fm,avg_accp_within_500m_to_1km_fm,
   total_accp_within_1_2km_fm,avg_accp_within_1_2km_fm,total_accp_with_more_than_2km_fm,avg_accp_with_more_than_2km_fm
   
   
   from set_one as xz1
   left join set_two as xz2
   on 
   xz1.yyyymmdd = xz2.yyyymmdd and
   xz1.test_ctrl =xz2.test_ctrl and 
   xz1.time_bucket = xz2.time_bucket and
   xz1.pickup_location_hex_8 =  xz2.pickup_location_hex_8    
    
   """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)

    print(i)
    per_Days_data=per_Days_data.append(df_test)


2023-07-04 00:00:00
2023-07-05 00:00:00
2023-07-06 00:00:00
2023-07-07 00:00:00
2023-07-08 00:00:00
2023-07-09 00:00:00
2023-07-10 00:00:00
2023-07-11 00:00:00
2023-07-12 00:00:00
2023-07-13 00:00:00
2023-07-14 00:00:00
2023-07-15 00:00:00
2023-07-16 00:00:00
2023-07-17 00:00:00
2023-07-18 00:00:00
2023-07-19 00:00:00
2023-07-20 00:00:00
2023-07-21 00:00:00
2023-07-22 00:00:00
2023-07-23 00:00:00


In [132]:
drt1=per_Days_data[~per_Days_data.test_ctrl.isnull()]#.to_clipboard()

In [148]:
len(drt1)

121468

In [111]:
drt1.to_clipboard()

In [147]:
drt1.iloc[110000:].to_clipboard()

In [126]:
drt1[drt1.yyyymmdd=='20230619'].to_clipboard()

In [151]:
drt1=drt1.reset_index(drop=True)

In [152]:
drt1.describe()

,Gross_Orders,Net_Orders,accepted_orders,total_map_riders,avg_map_riders,total_MR_within_500m,avg_MR_within_500m,total_MR_500m_to_1km,avg_MR_500m_to_1km,total_MR_1km_to_2km,...,total_MR_LP_caps,avg_MR_LP_caps,total_accp_within_500m_fm,avg_accp_within_500m_fm,total_accp_within_500m_to_1km_fm,avg_accp_within_500m_to_1km_fm,total_accp_within_1_2km_fm,avg_accp_within_1_2km_fm,total_accp_with_more_than_2km_fm,avg_accp_with_more_than_2km_fm
count,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,...,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000,121468.000000
mean,28.123391,3.892037,14.428582,2268.512316,42.580780,130.430418,2.498241,292.906971,6.160029,980.769775,...,956.123827,21.683002,6.453445,0.191228,14.611494,0.442406,21.257681,0.681653,1.862984,0.086736
std,82.626246,7.712091,27.358838,7331.759106,46.098957,642.386286,4.086093,874.954785,7.277115,2864.456673,...,2725.313065,25.638208,14.277271,0.217910,27.256143,0.314538,36.439601,0.298899,4.775396,0.177766
min,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,2.000000,73.000000,12.833333,3.000000,0.500000,9.000000,1.500000,34.000000,...,29.000000,5.386476,0.000000,0.000000,1.000000,0.181818,3.000000,0.500000,0.000000,0.000000
50%,7.000000,1.000000,5.000000,345.000000,28.084040,18.000000,1.375000,48.000000,3.800000,166.000000,...,158.000000,13.333333,2.000000,0.125000,5.000000,0.416667,8.000000,0.750000,0.000000,0.000000
75%,22.000000,4.000000,14.000000,1547.000000,56.000000,84.000000,3.110405,224.000000,8.062500,732.250000,...,712.000000,28.666667,6.000000,0.294118,16.000000,0.680068,23.000000,0.977778,2.000000,0.094595
max,6012.000000,192.000000,612.000000,528618.000000,525.705882,60050.000000,490.088235,35492.000000,99.673611,157677.000000,...,146035.000000,289.318182,376.000000,1.000000,654.000000,1.000000,756.000000,1.000000,222.000000,1.000000


In [170]:
v1=pd.DataFrame()

for i in pd.date_range('20230704','20230723'):

    query="""

          select t1.yyyymmdd,city_name,test_ctrl,segment,data_riderId,pickup_location_hex_8,t1.time_bucket,
          data_orderid,riderBusy_pings,riderreject_Pings,accepted_pings,already_accepted_pings,
          rider_accept_failure_pings,rider_ack_pings

          from
          (
                select yyyymmdd,data_riderId,pickup_location_hex_8,time_bucket,data_orderid,
                    --count(distinct data_orderid) as Gross_orders,
                    count(distinct case when event_type = 'rider_busy' then unique_ping_level_id end) as riderBusy_pings,
                    count(distinct case when event_type = 'rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
                    count(distinct case when event_type = 'rider_accepted' then unique_ping_level_id end) as accepted_pings,
                    count(distinct case when event_type = 'order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
                    count(distinct case when event_type = 'rider_accept_failed' then unique_ping_level_id end) as rider_accept_failure_pings,        
                    count(distinct case when event_type = 'rider_acknowledged' then unique_ping_level_id end) as rider_ack_pings
                from
                (
                       select yyyymmdd,quarter_hour,hh,epoch,data_propagationBatchId,data_riderId,data_orderid,
                        pickup_location_hex_8,event_type,unique_ping_level_id,time_bucket

                        from
                        (
                            select propagation_batch_id as data_propagationBatchId,rider_id as data_riderId,order_id as data_orderid,yyyymmdd,quarter_hour,hh,
                            concat(propagation_batch_id,rider_id,order_id) as unique_ping_level_id,pickup_location_hex_8,
                            updated_epoch as epoch,event_type,
                            (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
                                when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
                                when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
                                when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
                                else 'Rest_Evening' end) as time_bucket

                            from orders.dispatch_propagation_immutable
                            where yyyymmdd = cast({dt} as varchar)
                            --where yyyymmdd >= '20230715' --and yyyymmdd <= '20230707'
                            and order_id in (select distinct order_id from 
                                                        orders.order_logs_snapshot
                                                        -- where yyyymmdd = '20230625' 
                                                        where yyyymmdd = cast({dt} as varchar)
                                                        and service_obj_service_name = 'Auto'
                                                        and lower(trim(city_name)) in ('hyderabad','pune') )
                                                        
                            and event_type in ('rider_acknowledged')
                            and propagation_type in ('multicast')

                            union all

                            select data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,quarter_hour,hh,
                            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,pickup_location_hex_8,
                            updated_epoch as epoch,event_type,time_bucket


                            from(
                                    select event_type as eventtype,propagation_batch_id as data_propagationBatchId,rider_id as data_riderId,order_id as data_orderid,yyyymmdd,quarter_hour,hh,
                                    updated_epoch,event_type,pickup_location_hex_8,drop_location_hex_8,epoch,
                                    row_number() over(partition by propagation_batch_id,rider_id,order_id order by epoch desc) as row,
                                    (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
                                        when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
                                        when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
                                        when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
                                        else 'Rest_Evening' end) as time_bucket

                                    from orders.dispatch_propagation_immutable
                                        where yyyymmdd = cast({dt} as varchar)
                                        --where yyyymmdd >= '20230715' --and yyyymmdd <= '20230707'
                                        and order_id in (select distinct order_id from 
                                                        orders.order_logs_snapshot
                                                        -- where yyyymmdd = '20230625' 
                                                        where yyyymmdd = cast({dt} as varchar)
                                                        and service_obj_service_name = 'Auto'
                                                        and lower(trim(city_name)) in ('hyderabad','pune') )
                                        and propagation_type in ('multicast')
                                        and event_type in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                                ) base

                                where row =1
                            )                 
                        )
                        group by 1,2,3,4,5 

                    ) as t1

            left join (
                        select city_name,test_ctrl, hex_id ,time_bucket
                        from experiments_internal.auto_apr_fm_reduction_exp_230704 
                        where city_name in ('Pune','Hyderabad')

                     ) exp_dt
                     on t1.pickup_location_hex_8 = exp_dt.hex_id and t1.time_bucket = exp_dt.time_bucket

            left join(
                       select city,service,uid,segment
                        from experiments_internal.holdout_areas_v2
                        where service='Auto' and city in ('Hyderabad','Pune')

                   ) as v1 
                   on concat(t1.pickup_location_hex_8,'-',t1.time_bucket) = v1.uid



              where case when t1.yyyymmdd >='20230710' and concat(t1.pickup_location_hex_8,t1.time_bucket) 
                   in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                        '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                        '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                        '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                        '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                        '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                        '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                        '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                        '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                        '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                        '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                        '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                        '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                        '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                        '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                        '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                        '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                        '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                        '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                        '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                        '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                        '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                        '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                        '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                        '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                        '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                        '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                        '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                        '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                        '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                        '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                        '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                        '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                        '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                        '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                        '8860a24a65fffffmorning_peak'

                        ) then 1 else 0 end =0

                    and 
                       concat(t1.time_bucket,t1.pickup_location_hex_8) not in 
                        ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                        'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                        'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                        'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                        'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                        'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                        'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                        'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                        'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                        )


    """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)

    print(i)
    v1=v1.append(df_test)





2023-07-04 00:00:00
2023-07-05 00:00:00
2023-07-06 00:00:00
2023-07-07 00:00:00


OperationalError: Unexpected status code 503
b'no healthy upstream'

In [187]:
fda=v1[(v1.yyyymmdd=='20230707')&(v1.city_name=='Hyderabad')].groupby(['yyyymmdd','city_name','segment','test_ctrl','pickup_location_hex_8','time_bucket'],as_index=False)[['riderBusy_pings','riderreject_Pings','accepted_pings','already_accepted_pings','rider_accept_failure_pings','rider_ack_pings']].sum()




In [181]:
drt1.

,yyyymmdd,city_name,test_ctrl,pickup_location_hex_8,time_bucket,Gross_Orders,Net_Orders,accepted_orders,total_map_riders,avg_map_riders,...,total_MR_LP_caps,avg_MR_LP_caps,total_accp_within_500m_fm,avg_accp_within_500m_fm,total_accp_within_500m_to_1km_fm,avg_accp_within_500m_to_1km_fm,total_accp_within_1_2km_fm,avg_accp_within_1_2km_fm,total_accp_with_more_than_2km_fm,avg_accp_with_more_than_2km_fm
0,20230704,Hyderabad,ctrl_3,8860a24b39fffff,evening_peak,65,23,62,9883.0,89.036036,...,3178.0,28.630631,45.0,0.405405,89.0,0.801802,107.0,0.963964,2.0,0.018018
1,20230704,Hyderabad,ctrl_4,8860a25b19fffff,morning_peak,25,6,23,9328.0,160.827586,...,3410.0,58.793103,26.0,0.448276,49.0,0.844828,56.0,0.965517,1.0,0.017241
2,20230704,Hyderabad,ctrl_3,8860b52c2bfffff,afternoon,3,0,3,93.0,18.600000,...,47.0,9.400000,1.0,0.200000,2.0,0.400000,4.0,0.800000,1.0,0.200000
3,20230704,Pune,ctrl_1,886088508bfffff,morning_peak,11,2,7,1368.0,68.400000,...,1015.0,50.750000,7.0,0.350000,13.0,0.650000,16.0,0.800000,0.0,0.000000
4,20230704,Hyderabad,ctrl_3,8860a25811fffff,evening_peak,58,4,22,1638.0,22.135135,...,589.0,7.959459,6.0,0.081081,12.0,0.162162,28.0,0.378378,8.0,0.108108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121463,20230723,Hyderabad,ctrl_3,8860a25851fffff,afternoon,17,3,17,3850.0,142.592593,...,1586.0,58.740741,13.0,0.481481,25.0,0.925926,27.0,1.000000,1.0,0.037037
121464,20230723,Pune,ctrl_4,886088c9bdfffff,rest_morning,1,1,1,22.0,11.000000,...,20.0,10.000000,0.0,0.000000,1.0,0.500000,1.0,0.500000,0.0,0.000000
121465,20230723,Hyderabad,ctrl_3,8860a25a0dfffff,morning_peak,1,0,1,956.0,86.909091,...,398.0,36.181818,4.0,0.363636,8.0,0.727273,11.0,1.000000,0.0,0.000000
121466,20230723,Pune,test_2,88608858c3fffff,morning_peak,1,0,1,1475.0,39.864865,...,664.0,17.945946,35.0,0.945946,35.0,0.945946,35.0,0.945946,0.0,0.000000


In [171]:
one=pd.read_csv('pre_AOR_1.5 - Sheet11.csv')

In [173]:
two=pd.read_csv('post AOR_1.5 - Sheet11.csv')

In [176]:
combined=one.merge(two,on=['city_name','test_ctrl','pickup_location_hex_8','time_bucket'])

In [178]:
combined['AOR_diff']=combined.post_AOR - combined.pre_AOR

In [180]:
combined.to_csv('AOR_fm1.5_dt.csv',index=False)